# Entrega 1


In [1]:
import MySQLdb as mdb
import pandas as pd

con = mdb.connect("127.0.0.1", "guidecapture", "guidecapture", "guide_informe_final_corfo")
with con:
    cur = con.cursor()
    cur.execute("SELECT id, sequence, user_id, inittime, endtime FROM sessions;")
    rows = cur.fetchall()
    df = pd.DataFrame([[attribute for attribute in session] for session in rows])
    df.rename(columns={0:"id", 1:"sesión", 2:"usuario", 3:"inittime", 4:"endtime"}, inplace=True)

df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: context interface will be changed.  Use explicit conn.commit() or conn.rollback().
  """


,id,sesión,usuario,inittime,endtime
0,1,1 19,1,2016-05-12 10:12:04,2016-05-12 10:13:27
1,2,19,1,2016-05-12 10:15:17,2016-05-12 10:16:27
2,3,19 1 1 19 19,1,2016-05-12 10:18:28,2016-05-12 10:21:28
3,4,1,1,2016-05-12 11:03:31,2016-05-12 11:03:38
4,5,1 14 14 1 1 32 32,1,2016-05-12 11:07:12,2016-05-12 11:11:36


In [2]:
import numpy as np

# Transforms a session string into a dictionary containing the number of visits for each page in the session.
def total_visits(session):
    visits = session.split(" ")
    total_visits = {}
    for v in visits:
        if v in total_visits:
            total_visits[v] += 1
        else:
            total_visits[v] = 1
    return total_visits

# Distance function for the clustering algorithm.
def jaccard(s1, s2):
    s1_visits, s2_visits = total_visits(s1), total_visits(s2)
    all_pages = set(list(s1_visits.keys()) + list(s2_visits.keys()))
    intersection = 0
    union = len(all_pages)
    for key in all_pages:
        if key in s1_visits and key in s2_visits:
            intersection += 1
    return intersection / union

Dado que los datos son strings, debemos calcular manualmente nuestra matriz de distancia.

In [3]:
sessions = df['sesión']

DISTANCE_MATRIX = np.zeros((len(sessions), len(sessions)))

for index_1, session_1 in enumerate(sessions):
    print("{0:.2%}\r".format((index_1 + 1) / len(sessions)), end="")
    for index_2, session_2 in enumerate(sessions):
        if index_1 < index_2:
            DISTANCE_MATRIX[index_1, index_2] = 1 - jaccard(session_1, session_2)
        elif index_1 == index_2:
            DISTANCE_MATRIX[index_1, index_2] = 0
        else:
            continue

In [4]:
print(DISTANCE_MATRIX)

[[1.  0.5 1.  ... 0.5 1.  1. ]
 [0.5 1.  0.5 ... 0.  0.5 0.5]
 [1.  0.5 1.  ... 0.5 1.  1. ]
 ...
 [0.5 0.  0.5 ... 1.  0.5 0.5]
 [1.  0.5 1.  ... 0.5 1.  1. ]
 [1.  0.5 1.  ... 0.5 1.  1. ]]
